# 1. Import and Install Dependencies

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import time
import shutil
import torch

# WORD LIST

In [2]:
words = {}

In [3]:
words["adjective"]=[
"absent",
"beautiful",
"all",
"big",
"any",
"another",
"boring",
"deaf",
"dark",
"difficult",
"easy",
"full",
"fat",
"fast",
"hard",
"hot",
"less",
"low",
"loud",
"more",
"next",
"old",
"other",
"permanent",
"safe",
"same",
"slow",
"tall",
"short"]

In [4]:
words["adverb"]=[
"again",
"always",
"often",
"sometimes",
"through",
"up",
"together",
"where",
"why",
"when",
"how"
]

In [5]:
words["preposition"]=[
"about",
"among",
"at",
"around",
"before",
"below",
"down",
"from ",
"near",
"in",
"on",
"out",
"to",
"under",
"with",
"without"
]

In [6]:
words["pronoun"]=[
"he",
"her",
"hers",
"him",
"his",
"I",
"it",
"me",
"our",
"those",
"we",
"whose",
"who",
"you",
"your",
"us",
"what",
"them",
"my"
]

In [7]:
words["verb"]=[
"worry",
"wear",
"wait",
"try",
"touch",
"take",
"surprise",
"stop",
"start",
"spill",
"smile",
"shake",
"show",
"ride",
"remember",
"put",
"push",
"pour",
"press",
"play",
"pay",
"move",
"miss",
"love",
"laugh",
"kick",
"jump",
"keep",
"increase",
"hide",
"hate",
"go",
"give",
"forget",
"feel",
"enjoy",
"like",
"decrease",
"cry",
"hook",
"continue",
"close",
"open",
"call",
"carry",
"catch",
"bend",
"break",
"bath",
"run"
]

In [8]:
words["noun"]=[
"brother",
"family",
"freind",
"girkl",
"husvband",
"mother",
"mother in law",
"name",
"sister",
"wife",
"women",
"man",
"daiughter",
"son",
"children",
"cap",
"helment",
"pants",
"umbrella",
"t shirt",
"sweater",
"skirt",
"mask",
"shoes",
"dress",
"evening ",
"morning",
"afternoon",
"today",
"yesterday",
"tomorrow",
"night",
"late",
"time",
"bag",
"key",
"phone",
"ticket ",
"money",
"trash can",
"carrot",
"onion",
"potato",
"tomato",
"apple",
"banana",
"grape",
"guava",
"red",
"green",
"blue",
"yellow",
"orange",
"pink",
"brown",
"white",
"black",
"die",
"disease",
"ill",
"sick",
"wound",
"alive",
"cold",
"cough",
"gate",
"roof",
"bed",
"door",
"ladder",
"window",
"toilet",
]

# 2. Keypoints using MP Holistic

In [9]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [10]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [11]:
def draw_landmarks(image, results):
#     mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [12]:
def draw_styled_landmarks(image, results):
    # Draw face connections
#     mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
#                              mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
#                              mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
#                              ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 4. Setup Folders for Collection (Run only once)

In [13]:
os.mkdir("slr_data")
data_path="slr_data"
for grammar in words:
    os.mkdir(data_path+"/"+grammar)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'slr_data'

In [ ]:
os.mkdir("test_data")

# setup meta parameters

In [13]:
# Videos are going to be 30 frames in length
sequence_length = 24

In [14]:
# Thirty videos worth of data per word
no_sequences = 30

In [15]:
# Rest Time between each action recording
pause_time = 25

In [16]:
hcrop = 0
wcrop = 0

# action is the word variable, action="word"

In [113]:
#i'm using word "when" here
action="family"
#grammar is the grammar category the word is in:
#choose from ['adjective', 'adverb', 'preposition', 'pronoun', 'verb', 'noun']
grammar=""
for i in words:
    if action in words[i]:
        grammar=i
        break

# Begin capture for word (if something fails, just rerun cell, data is saved automatically)

# Press Q to quit while capturing. Word won't be saved

In [115]:
stop=False
nopose=torch.zeros(3,sequence_length,19)
noleft=torch.zeros(3,sequence_length,21)
noright=torch.zeros(3,sequence_length,21)

poselist=torch.zeros(3,sequence_length,19)
leftlist=torch.zeros(3,sequence_length,21)
rightlist=torch.zeros(3,sequence_length,21)
biglist=torch.zeros(no_sequences,3,sequence_length,61)
cap = cv2.VideoCapture(0)
bigresults=[]
tempresults=[]
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    sequence=0
    while(sequence<no_sequences):
        for frame_pause in range(pause_time):
            ret, frame = cap.read()
#             frame = frame[hcrop:-hcrop,wcrop:-wcrop]
#             print(frame.shape)
#             frame = cv2.resize(frame, (1280,960),interpolation=cv2.INTER_CUBIC)
            image, results = mediapipe_detection(frame, holistic)
            
            draw_styled_landmarks(image, results)
            image = cv2.flip(image,1)
            cv2.putText(image, 'STARTING COLLECTION', (250,300), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
            cv2.putText(image, str((pause_time-frame_pause)//10+1), (350,350), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
            cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            image = cv2.resize(image, (1280,960),interpolation=cv2.INTER_CUBIC)
            cv2.imshow('OpenCV Feed', image)
            if cv2.waitKey(10) & 0xFF == ord('n'):
                break
#             if cv2.waitKey(10) & 0xFF == ord('q'):
#                 stop=True
#                 break
        #record video of 32 frames
        tempresults = []
        for frame_num in range(sequence_length):
            ret, frame = cap.read()
#             frame = frame[hcrop:-hcrop,wcrop:-wcrop]
#             print(frame.shape)
#             frame = cv2.resize(frame, (1280,960),interpolation=cv2.INTER_CUBIC)
            image, results = mediapipe_detection(frame, holistic)
            tempresults.append(results)
            draw_styled_landmarks(image, results)
            image = cv2.flip(image,1)
            cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            cv2.putText(image, 'RECORDING', (640,650), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 5, cv2.LINE_AA)
            cv2.putText(image, str((sequence_length-frame_num)//10+1), (870,650), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 5, cv2.LINE_AA)
            
            image = cv2.resize(image, (1280,960),interpolation=cv2.INTER_CUBIC)
            cv2.imshow('OpenCV Feed', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                stop=True
                break
        if stop:
            break
        bigresults.append(tempresults)
        sequence+=1
    cap.release()
    cv2.destroyAllWindows()
    for sequence,results in enumerate(bigresults):
        for frame,result in enumerate(results):
            posepoints=result.pose_landmarks.landmark if result.pose_landmarks else nopose
            leftpoints=result.left_hand_landmarks.landmark if result.left_hand_landmarks else noleft
            rightpoints=result.right_hand_landmarks.landmark if result.right_hand_landmarks else noright
            if result.pose_landmarks:
                for i in range(len(posepoints)):
                    if(i>18):
                        break
                    poselist[0][frame][i]=posepoints[i].x
                    poselist[1][frame][i]=posepoints[i].y
                    poselist[2][frame][i]=posepoints[i].z
                poselist[0][frame][17]=posepoints[23].x; poselist[0][frame][18]=posepoints[24].x;
                poselist[1][frame][17]=posepoints[23].y; poselist[1][frame][18]=posepoints[24].y;
                poselist[2][frame][17]=posepoints[23].z; poselist[2][frame][18]=posepoints[24].z;
            else:
                poselist[0][frame]=torch.zeros(18)
                poselist[1][frame]=torch.zeros(18)
                poselist[2][frame]=torch.zeros(18)
            if result.left_hand_landmarks:
                for i in range(len(leftpoints)):
                    leftlist[0][frame][i]=leftpoints[i].x
                    leftlist[1][frame][i]=leftpoints[i].y
                    leftlist[2][frame][i]=leftpoints[i].z
            else:
                leftlist[0][frame]=torch.zeros(21)
                leftlist[1][frame]=torch.zeros(21)
                leftlist[2][frame]=torch.zeros(21)
            if result.right_hand_landmarks:
                for i in range(len(rightpoints)):
                    rightlist[0][frame][i]=rightpoints[i].x
                    rightlist[1][frame][i]=rightpoints[i].y
                    rightlist[2][frame][i]=rightpoints[i].z
            else:
                rightlist[0][frame]=torch.zeros(21)
                rightlist[1][frame]=torch.zeros(21)
                rightlist[2][frame]=torch.zeros(21)
                
        biglist[sequence]=torch.cat((leftlist,poselist,rightlist),2)

torch.save(biglist,"test_data/"+action+".pt")
torch.save(biglist,"slr_data/"+grammar+"/"+action+".pt")